In [ ]:
import pandas as pd

In [2]:
# Creating dummy columns from the 'genres' string
genre_dummies = merged_data['genres'].str.get_dummies(sep='|')

# Joining the new dummy columns
data = pd.concat([merged_data, genre_dummies], axis=1)

# Dropping the original 'genres' column 
data = data.drop('genres', axis=1)

# Display the first few rows with the new genre columns
data.head(3)

NameError: name 'merged_data' is not defined